In [1]:
import pandas as pd
from Classes import *
from General_Functions import *
import json 


data_dir_path:str = 'data/'

In [ ]:
all_actors = Actors_From_CSVs(data_dir_path + 'actor.csv', data_dir_path + 'actor_alias.csv')

### Testing MITRE lib

In [ ]:
import mitreattack.attackToExcel.attackToExcel as attackToExcel
import mitreattack.attackToExcel.stixToDf as stixToDf

In [ ]:
mitre:Mitre = Mitre(MitreDomains.ENTERPRISE)

In [ ]:
mitre.techniques_df.head()

In [ ]:
mitre.tactics_df.head()

In [4]:
import requests
from bs4 import BeautifulSoup
import re

df = pd.DataFrame(columns=["mitre_actor_id", "mitre_actor_name", "mitre_actor_groups", "mitre_actor_desc"])
URL = "https://attack.mitre.org/groups/" 
vals_to_remove = ["<td>","</td>"]
div_class = "table"

id_name_groups_pattern = r'<a[^>]*>(.*?)<\/a>'
desc_pattern = r'<p>(.*?)<\/p>'
df_rows = []

page = requests.get(URL)

print(f"STATUS: {page.status_code}")

html_content = str(page.content)
soup = BeautifulSoup(html_content, "html.parser")


table = soup.find("table")

rows = [str(s).split("\\n")[1:] for s in table.find_all("tr")]

i=0
for r in rows: 
    r = [s.strip() for s in r]
    r = [str(s) for s in r if not s in vals_to_remove]
    rows[i] = r
    i+=1

for r in rows[1:]: 
    this_df_row = []
    try: this_df_row.append(re.search(id_name_groups_pattern, r[0]).group(1).strip())
    except AttributeError: this_df_row.append([])
    
    try: this_df_row.append(re.search(id_name_groups_pattern, r[1]).group(1).strip())
    except AttributeError: this_df_row.append([])
    
    if r[2]: 
        this_df_row.append([str(s).strip() for s in r[2].split(",")])
    else: this_df_row.append([])
    
    try: this_df_row.append(re.search(desc_pattern, r[3]).group(1).strip())
    except AttributeError: this_df_row.append([])
    
    df_rows.append(this_df_row)
    

df = pd.DataFrame(df_rows, columns=["mitre_actor_id", "mitre_actor_name", "mitre_actor_groups", "mitre_actor_desc"])



STATUS: 200


In [5]:
df.head()

,mitre_actor_id,mitre_actor_name,mitre_actor_groups,mitre_actor_desc
0,G0018,admin@338,[],"<a href=""/groups/G0018"">admin@338</a> is a Chi..."
1,G0130,Ajax Security Team,"[Operation Woolen-Goldfish, AjaxTM, Rocket Kit...","<a href=""/groups/G0130"">Ajax Security Team</a>..."
2,G1000,ALLANITE,[Palmetto Fusion],"<a href=""/groups/G1000"">ALLANITE</a> is a susp..."
3,G0138,Andariel,[Silent Chollima],"<a href=""/groups/G0138"">Andariel</a> is a Nort..."
4,G1007,Aoqin Dragon,[],"<a href=""/groups/G1007"">Aoqin Dragon</a> is a ..."


In [27]:
for r in rows: print(r)

['<th scope="col">ID</th>', '<th scope="col">Name</th>', '<th scope="col">Associated Groups</th>', '<th scope="col">Description</th>', '</tr>']
['<a href="/groups/G0018"> G0018 </a>', '<a href="/groups/G0018"> admin@338 </a>', '', '<p><a href="/groups/G0018">admin@338</a> is a China-based cyber threat group. It has previously used newsworthy events as lures to deliver malware and has primarily targeted organizations involved in financial, economic, and trade policy, typically using publicly available RATs such as <a href="/software/S0012">PoisonIvy</a>, as well as some non-public backdoors. </p>', '', '</tr>']
['<a href="/groups/G0130"> G0130 </a>', '<a href="/groups/G0130"> Ajax Security Team </a>', 'Operation Woolen-Goldfish, AjaxTM, Rocket Kitten, Flying Kitten, Operation Saffron Rose', '<p><a href="/groups/G0130">Ajax Security Team</a> is a group that has been active since at least 2010 and believed to be operating out of Iran. By 2014 <a href="/groups/G0130">Ajax Security Team</a>